In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import seaborn as sns
import itertools as iter
import gc
from collections import Counter

In [3]:
products = pd.read_csv("Dataset/products.csv",dtype={
        'product_id': np.uint16,
        'product_name': object,
        'aisle_id': np.uint8,
        'department_id': np.uint8})

departments = pd.read_csv("Dataset/departments.csv",dtype={
        'department_id': np.uint16,
        'department_name': object})
aisles = pd.read_csv("Dataset/aisles.csv",dtype={
        'aisle_id': np.uint16,
        'aisle_name': object})
orders = pd.read_csv("Dataset/orders.csv", dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': object,
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})
order_products_train = pd.read_csv("Dataset/order_products__train.csv", dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})
order_products_prior = pd.read_csv("Dataset/order_products__prior.csv", dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})


In [4]:
items  = pd.merge(left =pd.merge(left=products, right=departments, how='left', on = 'department_id'), right=aisles, how='left', on = 'aisle_id')
prior_data = orders.merge(order_products_prior,how='inner', left_on='order_id', right_on='order_id').drop(columns=['eval_set'])
prior_data = prior_data.merge(items,how='inner', left_on='product_id', right_on='product_id')
train_data = orders.merge(order_products_train,how='inner', left_on='order_id', right_on='order_id').drop(columns=['eval_set'])
train_data = train_data.merge(items,how='inner', left_on='product_id', right_on='product_id')

In [5]:
ordersTrain=orders[orders['eval_set']=='train']
print(len(ordersTrain))
train_usr = pd.DataFrame(ordersTrain['user_id'].unique(), columns =['user_id'])
train_usr_sample =  train_usr.sample(2500, random_state=42).sort_values('user_id').reset_index(drop = True)
orders=orders[orders['user_id'].isin(train_usr_sample['user_id'])]
orders.head(5)

131209


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
4564,2006537,284,prior,1,1,12,NaN
4565,983957,284,prior,2,6,11,12.0
4566,2314173,284,prior,3,6,11,14.0
4567,106356,284,prior,4,2,9,10.0
4568,1801937,284,prior,5,2,14,14.0


In [6]:
#Creating Training Dataset
train_data = prior_data.merge(train_usr_sample,how='inner', left_on='user_id', right_on='user_id')
train_data['user_id'].nunique()

2500

In [7]:
order_product = train_data[['order_id','product_name']]
order_product.head()

,order_id,product_name
0,1422524,Soda
1,3306294,Soda
2,48682,Soda
3,2214144,Soda
4,903609,Soda


In [8]:
order_product['product_number'] = order_product.groupby(['order_id']).cumcount()+1
order_product = order_product.sort_values(by=['order_id','product_number'])
order_product.head()

C:\Users\Dipesh\AppData\Local\Temp\ipykernel_6928\242704435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_product['product_number'] = order_product.groupby(['order_id']).cumcount()+1


,order_id,product_name,product_number
62596,56,Organic Hass Avocado,1
62608,56,Organic Baby Spinach,2
62716,56,Large Alfresco Eggs,3
62740,56,Shredded Parmesan,4
62814,56,Blueberries,5


In [9]:
order_product = order_product.pivot(index='order_id', columns='product_number', values='product_name')
order_product.head()

product_number,1,2,3,4,5,6,7,8,9,10,...,75,76,77,78,79,80,81,82,83,84
order_id,,,,,,,,,,,,,,,,,,,,,
56,Organic Hass Avocado,Organic Baby Spinach,Large Alfresco Eggs,Shredded Parmesan,Blueberries,Organic Garlic,Organic Tomato Paste,Buttermilk Biscuits,Orange Juice,Coconut Almond Creamer Blend,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,Bag of Organic Bananas,Seedless Red Grapes,Apple Cinnamon GoGo Squeez,Organic AppleApple,"YoKids Squeezers Organic Low-Fat Yogurt, Straw...",Whole Milk,Organic Fuji Apple,Natural Artisan Water,Chocolate Chip Cookie Dough Ice Cream,Real Aged Cheddar Macaroni & Cheese,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,Original Hummus,White Cheddar Macaroni & Cheese Cups,Cold Pressed Watermelon & Lemon Juice Blend,Paleo Blueberry Muffin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,Honeycrisp Apple,Brussels Sprouts,Organic Garnet Sweet Potato (Yam),Organic Bosc Pear,Yukon Gold Potatoes 5lb Bag,Fresh Cauliflower,Large Lemon,Organic Unsalted Butter,Organic Banana,Italian Sparkling Mineral Water,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,Organic Strawberries,Pure Sparkling Water,Organic Pumpkin Puree,Yobaby Organic Plain Yogurt,Orange Sparkling Water,Organic Pure 85% Dark Chocolate Bar,Organic Sprouted English Muffin,Organic Grass-Fed Classic Mild Cheddar Macaron...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
op_index = order_product.index.values
product_list =  order_product.apply(lambda x :'*_*'.join(x.fillna("xxx").astype(str)),1)
product_list.index = op_index
order_product['product'] = product_list
order_product

product_number,1,2,3,4,5,6,7,8,9,10,...,76,77,78,79,80,81,82,83,84,product
order_id,,,,,,,,,,,,,,,,,,,,,
56,Organic Hass Avocado,Organic Baby Spinach,Large Alfresco Eggs,Shredded Parmesan,Blueberries,Organic Garlic,Organic Tomato Paste,Buttermilk Biscuits,Orange Juice,Coconut Almond Creamer Blend,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Hass Avocado*_*Organic Baby Spinach*_*...
105,Bag of Organic Bananas,Seedless Red Grapes,Apple Cinnamon GoGo Squeez,Organic AppleApple,"YoKids Squeezers Organic Low-Fat Yogurt, Straw...",Whole Milk,Organic Fuji Apple,Natural Artisan Water,Chocolate Chip Cookie Dough Ice Cream,Real Aged Cheddar Macaroni & Cheese,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bag of Organic Bananas*_*Seedless Red Grapes*_...
154,Original Hummus,White Cheddar Macaroni & Cheese Cups,Cold Pressed Watermelon & Lemon Juice Blend,Paleo Blueberry Muffin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Original Hummus*_*White Cheddar Macaroni & Che...
210,Honeycrisp Apple,Brussels Sprouts,Organic Garnet Sweet Potato (Yam),Organic Bosc Pear,Yukon Gold Potatoes 5lb Bag,Fresh Cauliflower,Large Lemon,Organic Unsalted Butter,Organic Banana,Italian Sparkling Mineral Water,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Honeycrisp Apple*_*Brussels Sprouts*_*Organic ...
216,Organic Strawberries,Pure Sparkling Water,Organic Pumpkin Puree,Yobaby Organic Plain Yogurt,Orange Sparkling Water,Organic Pure 85% Dark Chocolate Bar,Organic Sprouted English Muffin,Organic Grass-Fed Classic Mild Cheddar Macaron...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Strawberries*_*Pure Sparkling Water*_*...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420559,Icelandic Style Skyr Blueberry Non-fat Yogurt,Organic Romaine,Uncured Pepperoni,"Yogurt, Lowfat, Strawberry",Shredded Mozzarella,Spring Water,Organic Raspberry Lowfat Yogurt,Carrot And Celery Sticks,Perrier Lime,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Icelandic Style Skyr Blueberry Non-fat Yogurt*...
3420645,Gluten Free Mediterranean Crackerz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gluten Free Mediterranean Crackerz*_*xxx*_*xxx...
3420650,Organic Unsweetened Vanilla Almond Milk,Organic Baby Spinach,Organic Lacinato (Dinosaur) Kale,Sparkling Clementine Juice,Grapefruit Sparkling Juice Beverage,Wild Caught Frozen Sockeye Salmon Fillet,Gluten Free Chocolate Chip Cookies,Toasted Coconut Almondmilk Blend,Total 2% All Natural Low Fat 2% Milkfat Greek ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organic Unsweetened Vanilla Almond Milk*_*Orga...


In [11]:
order_product['order_id'] = order_product.index
order_product = order_product[['order_id','product']]
order_product.head()

product_number,order_id,product
order_id,,
56,56,Organic Hass Avocado*_*Organic Baby Spinach*_*...
105,105,Bag of Organic Bananas*_*Seedless Red Grapes*_...
154,154,Original Hummus*_*White Cheddar Macaroni & Che...
210,210,Honeycrisp Apple*_*Brussels Sprouts*_*Organic ...
216,216,Organic Strawberries*_*Pure Sparkling Water*_*...


In [12]:
op = order_product.values
for i in range(len(op)):
    l = list(set(op[i][1].split('*_*')))
    if ('xxx' in l):
        l.remove('xxx')
    l = list(iter.combinations(l,2))
    op[i][1] = l

In [13]:
op_df = pd.DataFrame(op, columns=['order_id','product_pairs'])
op_df.head()

,order_id,product_pairs
0,56,"[(Buttermilk Biscuits, Blueberries), (Buttermi..."
1,105,"[(Organic AppleApple, Cage Free Large White Eg..."
2,154,"[(Original Hummus, Paleo Blueberry Muffin), (O..."
3,210,"[(Honeycrisp Apple, Organic Garnet Sweet Potat..."
4,216,"[(Orange Sparkling Water, Organic Pumpkin Pure..."


In [14]:
# del products
del departments
del aisles
del orders
del order_products_train
del order_products_prior
del items
del order_product
del prior_data

In [15]:
import gc
gc.collect()

36

In [16]:
res = op_df.apply(lambda x: pd.Series(x['product_pairs']),axis=1).stack().reset_index(level=1, drop=True)
res.name = 'product_pair'

C:\Users\Dipesh\AppData\Local\Temp\ipykernel_6928\2479172969.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  res = op_df.apply(lambda x: pd.Series(x['product_pairs']),axis=1).stack().reset_index(level=1, drop=True)


In [17]:
op_df = op_df.drop('product_pairs', axis=1).join(res)
op_df.head()

,order_id,product_pair
0,56,"(Buttermilk Biscuits, Blueberries)"
0,56,"(Buttermilk Biscuits, Organic Baby Spinach)"
0,56,"(Buttermilk Biscuits, Organic Garlic)"
0,56,"(Buttermilk Biscuits, Coconut Almond Creamer B..."
0,56,"(Buttermilk Biscuits, Orange Juice)"


In [18]:
op_df['order_id']=op_df['order_id'].astype('int64')

In [19]:
del res
gc.collect()

654263

In [20]:
train_data = train_data[['user_id','order_id']]
train_data.head()

,user_id,order_id
0,4028,1422524
1,4028,3306294
2,4028,48682
3,4028,2214144
4,4028,903609


In [21]:
up_df = train_data.merge(op_df, how='inner', on='order_id')

In [22]:
up_df = up_df.dropna()

In [23]:
del op_df
del train_data
gc.collect()

72

In [24]:
up_df.head()
up_df.shape

(64658133, 3)

In [25]:
up_df['user_id'].nunique()

2488

In [26]:
users = list(set(up_df['user_id']))
user_set1 = users[0:311*1]
user_set2 = users[311*1:311*2]
user_set3 = users[311*2:311*3]
user_set4 = users[311*3:311*4]
user_set5 = users[311*4:311*5]
user_set6 = users[311*5:311*6]
user_set7 = users[311*6:311*7]
user_set8 = users[311*7:311*8]

In [27]:
up_df1 = up_df[up_df['user_id'].isin(user_set1)]
up_df2 = up_df[up_df['user_id'].isin(user_set2)]
up_df3 = up_df[up_df['user_id'].isin(user_set3)]
up_df4 = up_df[up_df['user_id'].isin(user_set4)]
up_df5 = up_df[up_df['user_id'].isin(user_set5)]
up_df6 = up_df[up_df['user_id'].isin(user_set6)]
up_df7 = up_df[up_df['user_id'].isin(user_set7)]
up_df8 = up_df[up_df['user_id'].isin(user_set8)]

In [28]:
del user_set1
del user_set2
del user_set3
del user_set4
del user_set5
del user_set6
del user_set7
del user_set8